In [1]:
#Import the libraries we’ll be using throughout our notebook:
import pandas as pd
import numpy as np
import spacy
from tqdm import tqdm
import re
import torch

In [2]:
#Read and Inspect the Data
News = pd.read_csv("bloomberg_marked_df.csv")
News.shape

(1912, 3)

In [3]:
News.head()

,mark,date,news
0,1,2021-07-22,India’s Environment Ministry has almost halved...
1,1,2021-07-22,Singapore’s home price growth slowed in the se...
2,1,2021-07-22,"What keeps breaking the buck, is engaged in fi..."
3,1,2021-07-22,Malaysia has confirmed one case of an individu...
4,1,2021-07-22,"Xponential Fitness Inc., the franchise owner o..."


In [4]:
#Text Cleaning and Preprocessing
# remove URL's from train and test
News['news'] = News['news'].apply(lambda x: re.sub(r'http\S+', '', x))

In [5]:
# remove punctuation marks
punctuation = '!"#$%&()*+-/:;<=>?@[\\]^_`{|}~'
News['news'] = News['news'].apply(lambda x: ''.join(ch for ch in x if ch not in set(punctuation)))

# convert text to lowercase
News['news'] = News['news'].str.lower()

# remove numbers
News['news'] = News['news'].str.replace("[0-9]", " ")

# remove whitespaces
News['news'] = News['news'].apply(lambda x:' '.join(x.split()))


<ipython-input-5-567b2ff1cea7>:9: FutureWarning: The default value of regex will change from True to False in a future version.
  News['news'] = News['news'].str.replace("[0-9]", " ")


In [6]:
#clean text
News.head()

,mark,date,news
0,1,2021-07-22,india’s environment ministry has almost halved...
1,1,2021-07-22,singapore’s home price growth slowed in the se...
2,1,2021-07-22,"what keeps breaking the buck, is engaged in fi..."
3,1,2021-07-22,malaysia has confirmed one case of an individu...
4,1,2021-07-22,"xponential fitness inc., the franchise owner o..."


In [7]:
#Preparing ELMo Vectors
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
tf.reset_default_graph()



#elmo = hub.Module("https://tfhub.dev/google/elmo/3")
elmo = hub.Module("https://tfhub.dev/google/elmo/3", trainable=True)

Instructions for updating:
non-resource variables are not supported in the long term


In [8]:
def elmo_vectors(x):
    embeddings = elmo(x.tolist(), signature="default", as_dict=True)["elmo"]
     
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        sess.run(tf.tables_initializer())
        # return average of ELMo features
        return sess.run(tf.reduce_mean(embeddings,1))

In [9]:
list_news = [News[i:i+100] for i in range(0,News.shape[0],100)]

In [10]:
# Extract ELMo embeddings
elmo_news = [elmo_vectors(x['news']) for x in list_news]

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


...

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [ ]:
elmo_train_new = np.concatenate(elmo_news, axis = 0)

In [31]:
data = {'news': elmo_train_new}

In [32]:
Datafr = pd.DataFrame(data)

In [33]:
Datafr

,news
0,"[-0.0760562, -0.0432576, 0.03269349, -0.049461..."
1,"[-0.032277316, -0.1418127, 0.044348404, -0.011..."
2,"[0.05199899, -0.030011326, 0.103910156, 0.1526..."
3,"[-0.054891933, -0.0064048804, 0.009097336, -0...."
4,"[-0.024295278, -0.02294216, 0.03299467, -0.034..."
5,"[0.0034245558, -0.010729793, -0.0033818968, -0..."
6,"[-0.0760562, -0.0432576, 0.03269349, -0.049461..."
7,"[-0.008383959, -0.008444629, 0.08218849, -0.02..."
8,"[-0.03506169, -0.021230124, 0.02670655, -0.013..."
9,"[-0.015689364, -0.0051695406, 0.017227866, -0...."


In [36]:
Datafr.to_csv('bloomberg_marked_elmo.csv', sep='\t')